In [ ]:
!pwd

In [ ]:
# Setting lerobot env
%cd /workspace
!git clone https://github.com/huggingface/lerobot.git
%cd lerobot

from lerobot_setup import patch_pyproject

patch_pyproject()

!git diff

# Install the library in editable mode with training dependencies
import sys

!apt update && apt install -y ffmpeg
print(f"executable {sys.executable}")
#!{sys.executable} -m pip install "torch==2.10.0" \
#    "torchvision>=0.21.0" \
#    "torchcodec==0.10.0" \
#    "transformers==4.57.6" \
#    --extra-index-url https://download.pytorch.org/whl/cu130
!{sys.executable} -m pip install -e .
!{sys.executable} -m pip install wandb
!{sys.executable} -m pip install python-dotenv

In [ ]:
# Setting up supporting env, e.g., huggingface and wandb tokens
import os
import sys
from dotenv import load_dotenv

load_dotenv(override=True)

def get_secret(key_name):
    # 1. Try Google Colab userdata (Secret Manager)
    if 'google.colab' in sys.modules:
        try:
            from google.colab import userdata
            return userdata.get(key_name)
        except Exception:
            pass # Fallback if secret not found in Colab
            
    # 2. Try Local/WSL .env file
    load_dotenv(override=True)
    
    # 3. Fallback to standard environment variable (os.environ)
    return os.getenv(key_name)

# Usage
HF_TOKEN = get_secret('HF_TOKEN')
WANDB_API_KEY = get_secret('WANDB_API_KEY')

# Apply to environment so libraries find them automatically
if HF_TOKEN:
    # HUGGINGFACE env
    os.environ['HF_TOKEN'] = HF_TOKEN
    from huggingface_hub import login
    login(token=HF_TOKEN)
    print("login to hf")
else:
    print("login to hf failed")

if WANDB_API_KEY:
    os.environ['WANDB_API_KEY'] = WANDB_API_KEY
    os.environ["WANDB_NOTEBOOK_NAME"] = "train_so101_model.ipynb"
    # WANDB env
    import wandb
    wandb.login()
    print("login to wandb")
else:
    print("login to wandb failed")

In [ ]:
%cd /workspace/lerobot
# Training ACT model.
# Adjust the following variables before starting training
tag = '400episodes'
policy_type = 'diffusion'
n_action_steps = 10 if policy_type == 'diffusion' else 100
hf_namespace = 'fanaustinca'
training_repo = f"{hf_namespace}/fll_training_combined"
model_repo = f"{hf_namespace}/fll_diffusion_model_400episodes_v2"
# Use 'local_output' consistently
local_output = f"outputs/model/{model_repo}"
steps = 150000
batch_size = 64
num_workers = 16

# Clean up output from last run
!rm -rf {local_output}

tfs_config = {
  "to_tensor": {"type": "ToImage"},
  "affine": {
    "type": "RandomAffine",
    "kwargs": {"degrees": 0, "translate": [0.05, 0.05]}
  }
}

"""
    --dataset.image_transforms.enable=true \
    --dataset.image_transforms.random_order=false \
    --dataset.image_transforms.tfs='{tfs_string}' \
"""

# Convert to a compact JSON string for the CLI
import json
tfs_string = json.dumps(tfs_config, separators=(',', ':'))

arguments = f"""\
    --policy.type={policy_type} \
    --dataset.repo_id={training_repo} \
    --job_name=fll_training_{policy_type} \
    --output_dir={local_output} \
    --policy.n_action_steps={n_action_steps} \
    --policy.device=cuda \
    --steps={steps} \
    --save_freq=5000 \
    --batch_size={batch_size} \
    --log_freq=100 \
    --num_workers={num_workers} \
    --policy.use_amp=true \
    --policy.push_to_hub=true \
    --policy.repo_id={model_repo} \
    --dataset.image_transforms.enable=true \
    --dataset.image_transforms.random_order=true \
    --policy.num_inference_steps=30 \
    --wandb.enable=true

    --robot.type=so101_follower \
    --robot.port=/dev/ttyACM1 \
    --robot.cameras='{
      "wrist": {"type": "opencv", "index_or_path": 2, "fps": 30, "width": 640, "height": 480}, 
      "top": {"type": "opencv", "index_or_path": 4, "fps": 30, "width": 640, "height": 480}
    }' \
    --policy.type={policy_type} \
    --policy.pretrained_path="./my_model/fll_act_model_local" \
    --dataset.repo_id=fanaustinca/eval_act_260205_v2 \
  --dataset.num_episodes=10 \
  --dataset.single_task="fll_mission" \
  --display_data true

"""
print(f"{sys.executable} src/lerobot/scripts/lerobot_record.py {arguments}")

#!{sys.executable} src/lerobot/scripts/lerobot_train.py {arguments}

#!hf repo tag create {model_repo} {tag} -m "Diffusion model trained on {training_repo}, tag {tag}"